<a href="https://colab.research.google.com/github/alexaleddy/2025_AST4300_Project3/blob/main/Task_35_Figure_5_Alexa_Leddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import some things
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import fits, ascii
from astropy.table import Table
from astropy.coordinates import SkyCoord

In [ ]:
#plot stuff

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['axes.titlesize'] = 14 # font size for plot titles
plt.rcParams['axes.labelsize'] = 12 # font size for x and y axis labels
plt.rcParams['xtick.labelsize'] = 10 # font size for x-axis tick labels
plt.rcParams['ytick.labelsize'] = 10 # font size for y-axis tick labels
plt.rcParams['legend.fontsize'] = 10 # font size for legend

In [ ]:
#potential names:

#comp['Age_seis']     : reference seismic ages
#comp['Age_ours']     : our NN ages
#comp['Age_astroNN']  : astroNN ages (literature)
#comp['Age_StarFlow'] : StarFlow ages (literature)

age_ref   = np.array(comp['Age_seis'])
age_ours  = np.array(comp['Age_ours'])

ml_methods = [
    {
        "col": "Age_astroNN",
        "label": "astroNN",
        "color": "r"
    },
    {
        "col": "Age_StarFlow",
        "label": "StarFlow",
        "color": "c"
    }
]

In [ ]:
#multi panel figure: for each method, 1:1 plot (ours vs literature) + histogram of age differences

fig, axes = plt.subplots(
    nrows=len(ml_methods), ncols=2,
    figsize=(10, 4*len(ml_methods)),
    constrained_layout=True
)

if len(ml_methods) == 1:
    axes = np.array([axes])

for i, method in enumerate(ml_methods):
    lit_age = np.array(comp[method["col"]])

    #only finite, positive ages
    finite = (
        np.isfinite(age_ref) & np.isfinite(age_ours) & np.isfinite(lit_age) &
        (age_ref > 0) & (age_ours > 0) & (lit_age > 0)
    )
    ref   = age_ref[finite]
    ours  = age_ours[finite]
    lit   = lit_age[finite]


In [ ]:
#Left: 1:1 comparison plot

    ax1 = axes[i, 0]
    ax1.scatter(lit, ours, s=4, alpha=0.5, color=method["color"],
                label=f"Ours vs {method['label']}")
    ax1.plot([0, 14], [0, 14], 'k-', linewidth=1.5, label="1:1")

    ax1.set_xlim(0, 14)
    ax1.set_ylim(0, 14)
    ax1.set_ylabel("Our NN Age [Gyr]")
    if i == len(ml_methods) - 1:
        ax1.set_xlabel(f"{method['label']} Age [Gyr]")

    ax1.set_title(f"Our NN vs {method['label']}")
    ax1.legend(loc="upper left")

In [ ]:
#Right: histogram of age differences

    ax2 = axes[i, 1]
    diff = ours - lit

    ax2.hist(diff, bins=40, color=method["color"], alpha=0.7)
    ax2.axvline(0, color='k', linestyle='--', linewidth=1)

    mean_diff = np.mean(diff)
    std_diff  = np.std(diff)
    ax2.text(
        0.05, 0.95,
        f"mean = {mean_diff:.2f} Gyr\nσ = {std_diff:.2f} Gyr",
        transform=ax2.transAxes,
        va='top',
        ha='left'
    )

    ax2.set_ylabel("Number of stars")
    if i == len(ml_methods) - 1:
        ax2.set.xlabel("Our NN Age − Literature Age [Gyr]")
    ax2.set_title(f"Age Difference: Ours − {method['label']}")

plt.suptitle("Fig 5: Comparison of Machine Learning Age Methods", y=1.02)
plt.show()
